In [1]:

### Table of Contents

* [1. Introduction](#1.introduction)
* [2. First basic problem](#fbp)
    * [2.1. Math and first step with pyomo for solving the problem](#math)
    * [2.2. Variables ](#variables)
    * [2.3. Constraints](#constraints)
* [3. Extensions of this operation problem ](#temporal)
    * [3.1. Linear temporal coupling with ramp constraints ](#ramp)
    * [3.2. Linear spatial coupling with spatial constraints - Problem Op3 Multi-Area -](#spatial)
* [4. Storage operation ](#storage)
    * [4.1. Optimisation of a storage market participation](#storageMarket)
    * [4.2. Simultaneous optimisation of storage and electric system](#storagecoupling)
* [5. Complete French case ](#France)

## 1. Introduction <a class="anchor" id="1.introduction"></a>

This document will gives a chance to understand
 - how to do a simulation of the hourly operation of an electric system for a year
 - understand the mathematical formulation of the optimisation problem
 - learn to analyse de results of the optimisation and in particular the Lagrange multipliers
 - get in touch with pyomo (a python package to write optimisation problems)

It proposes to enter the subject by increasing progressively
the number of variables and constraints in the optimisation problem, hence moving toward more realism through the document, introducing:
 - ramp constraints that implies a simple temporal coupling
 - spatially indexed variables and congestion constraints that implies a simple spatial coupling.
 - storage constraints that implies a temporal coupling

It relies on different test cases that allow to
 - consider different production means (nuclear, thermal, solar, onshore wind power, offshore wind power, hydro, curtailement of consumption, storage)
 - consider different meteorological years for France
 - consider different countries in the multi-zone case (France, Germany, GB, Spain)

If, after reading this file, you want to build your own pyomo model you can create it in f_operationModels.py by adding a function that mimics the other ones. You can create you test case by adding a case_XXX.py and/or case_XXX.ipynb file in this folder.

## 2. First basic problem <a class="anchor" id="fbp"></a>
In this section we propose a simple version of the optimisation problem with a mathematical description.
This allows us to discuss the use of pyomo (a python package to write optimisation problems).
### 2.1. Math and first step with pyomo for solving problem<a class="anchor" id="math"></a>

**Mathematical formulation as a linear programming problem**

\begin{align}
&\text{Cost function }& &\min_{x}  \sum_t \sum_i \pi_i x_{it}\;\;\; & & \pi_i \text{ marginal cost}\\
&\text{Power limit }   & &\text{ s.t.} \;\; 0 \leq x_{it}\leq a_{it} \bar{x_i} & &\bar{x_i} \text{ installed power, }  a_{it} \text{ availability}\\
&\text{Meet demand }   & & \sum_i x_{it} \geq  C_t  && C_t \text{ Consumption}\\
&\text{Stock limit }   & &\sum_t x_{it}\leq E_i && E_i=\bar{x_i}*N_i \text{ Energy capacity limit}\\
\end{align}

This is linear programing and could be transformed into something like

$
\begin{align}
& \min_y & c^Ty \\
& Ay\leq b
\end{align}
$

 with a well chosen parameter matrix $A$, parameter vector $b$ and $c$ and variable vector $y$. While increasing complexity of the problem,
 this can become very painful.


With the Python package Pyomo it is almost sufficient to write the mathematical equations. Pyomo is then charged of building the matrix form,
and you just have to think about problem formulation. Pyomo is similar to other tools such as GAMS, AMPL, ...


**Principles of a pyomo model**
To build a model, Pyomo needs three different kinds of data : sets, parameters and variables.

 - *Sets* are dimensions, here the time and the name of technology plus a mix of these two :
Date, TECHNOLOGIES and Date_TECHNOLOGIES (product set).

 - *Parameters* are tables indexed by set whose values are specified by the user.
Here is the list of the parameters we use in the simplest cases : energycost, EnergyNbhourCap, capacity,
availability factor, area consumption.

 - *Variable* are tables indexed by set whose values are found by the solver :
the energy produced by each mean of production.

Now if you wish to learn more about pyomo and to see how the optimisation problem is writen in pyomo language,
take look at function GetElectricSystemModel_GestionSingleNode  (by doing a control-click on it in the code that follos a bit later,
otherwise it is in f_operationModels.py).
We will use this function in the following exemple.

**First simulation -- description of case_step_by_step_learning**

If you want to give it a try, use the case_step_by_step_learning.py or case_step_by_step_learning.ipynb.
There, we start with a simple test case, with only two production means (nuke and thermal) but you can run
this with more than that, and change the installed power. If you want to change assumptions you will need to
change the csv files that are loaded below.

## 2.2 Analysing results : lagrange multipliers <a class="anchor" id="optiofope"></a>

Lagrange multiplier might be more difficult to understand for those who still lack a good optimisation course (the one at second semester of first year of MINES ParisTech is perfect).
In cases you want to dig this, you can have a look at Boyd's course, e.g. starting with [lecture 8](https://www.youtube.com/watch?v=FJVmflArCXc) or before.
The most important message here is that lagrange multipliers associated to the demand constraint (here called 'energyCtr') are meant to mimic market prices.
Lagrange multiplier associated to this constraint at time t is the marginal cost that one would pay to increase $C_t$ by a small amount. They can be used to simulate market prices.

## 3. Extensions of this operation problem <a class="anchor" id="temporal"></a>
### 3.1.  Linear temporal coupling with ramp constraints <a class="anchor" id="ramp"></a>
In the this section, we will increase the complexity of the problem
given in Section 2 and add : dependency on area z (country),
 a congestion constraint, ramp constraints.

\begin{align}
&\text{Cost function }& &\min_{x}  \sum_z \sum_t \sum_i \pi_{iz} x_{itz}\;\;\; & & \pi_{iz} \text{ marginal cost}\\
&\text{Power limit }   & &\text{ s.t.} \;\; 0 \leq x_{itz}\leq a_{itz} \bar{x_{iz}} & &\bar{x_{iz}} \text{ installed power, }  a_{itz} \text{ availability}\\
&\text{Meet demand }   & & \sum_i x_{itz} \geq  C_{tz}  && C_{tz} \text{ Consumption}\\
&\text{Stock limit }   & &\sum_t x_{it}\leq E_i && E_i=\bar{x_i}*N_i \text{ Energy capacity limit}\\
&\text{ramp limit }   & &rc^-_i *x_{it}\leq x_{it}-x_{i(t+1)}\leq rc^+_i *x_{it} && rc^+_i rc^-_i\text{ ramp limit}\\
\end{align}

## 4. Storage operation <a class="anchor" id="storage"></a>
### 4.1. Optimisation of a storage market participation <a class="anchor" id="storageMarket"></a>
Just have a look at optim-Storage.ipynb

### 4.2. Simultaneous optimisation of storage and electric system <a class="anchor" id="storagecoupling"></a>

The optimisation problème is the same as before. Is this section, we only add the contraints regarding the storage :

\begin{align}
&\text{Cost function }& &\min_{x}  \sum_t \sum_i \pi_i x_{it}\;\;\; & & \pi_i \text{ marginal cost}\\
&\text{Power limit }   & &\text{ s.t.} \;\; 0 \leq x_{it}\leq a_{it} \bar{x_i} & &\bar{x_i} \text{ installed power, }  a_{it} \text{ availability}\\
&\text{Meet demand }   & & \sum_i x_{it}+\sum_j (z_{out_{jt}}-z_{in_{jt}}) \geq  C_t  && C_t \text{ Consumption}, z_{jt} \text{ storage}\\
&\text{Stock limit }   & &\sum_t x_{it}\leq E_i && E_i=\bar{x_i}*N_i \text{ Energy capacity limit}\\
\end{align}

Adding several storages to this operation problem is done here :
\begin{align}
&\text{storage power constraint}& &0\leq z_{in/out_{jt}}\leq p_{\max} \\
&\text{storage energy constraint}& &0\leq \sum_{k=1}^t \left( z_{in_{jk}}.\eta_{in_j}-\frac{z_{out_{jk}}}{\eta_{out_j}} \right)\leq c_{\max} && \eta_j \text{ Storage efficiency (in/out)} \\
\end{align}

By introducing a new variable representing the storage level at any time t, the last equation become :
\begin{align}
\text{storage energy constraint}& &0\leq  s_{jt} \leq c_{\max} \\
\text{where}& & s_{jt} = s_{j,t-1}.(1-\delta_j) + z_{in_{jk}}.\eta_{in_j}-\frac{z_{out_{jk}}}{\eta_{out_j}} && \delta_j \text{ Storage dissipation}\\
\end{align}

This model is implemented in function GetElectricSystemModel_GestionSingleNode_with1Storage in file
f_operationModels.py. The idea of this modelis quite general and a version exists also
in the multi-node case and for planing problems (in file Planing_optimisation/f_planingModels.py)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2719137325.py, line 3)